<a href="https://colab.research.google.com/github/jameswu811102/Train_Your_Custom_Yolo_v4/blob/main/YOLOv4_tiny_(Custom).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 掛載 google colab

In [ ]:
from google.colab import drive
drive.mount("/content/GDrive")

# 複製 darknet repository


In [ ]:
!git clone https://github.com/AlexeyAB/darknet

# 開啟 GPU 及 OpenCV的使用

In [ ]:
%cd /content/darknet

!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile

# 如果是即時偵測須加上
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

# 編譯

In [ ]:
# 編譯
!make

# 客製化一些功能

In [ ]:
# 將預測照片在colab中印出
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(20, 15)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# 上傳檔案用的功能
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# 下載檔案用  
def download(path):
  from google.colab import files
  files.download(path)

---------------------------------------------------------------------------
---------------------------------------------------------------------------
---------------------------------------------------------------------------

---------------------------------------------------------------------------
---------------------------------------------------------------------------
---------------------------------------------------------------------------

---------------------------------------------------------------------------
---------------------------------------------------------------------------
---------------------------------------------------------------------------

# 如何訓練自己的Yolo tiny物件辨識器






## 0. 下載以事先訓練好的權重係數檔案 `yolov4-tiny.conv.29`

In [ ]:
!mkdir -p /content/darknet/custom_yolo
!mkdir -p /content/darknet/custom_yolo/backup
!mkdir -p /content/darknet/custom_yolo/cfg
!mkdir -p /content/darknet/custom_yolo/data
!mkdir -p /content/darknet/custom_yolo/datasets

In [ ]:
# 下載事先訓練好的卷積網絡權重檔案
!wget -P /content/darknet/custom_yolo https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

# 1. 將你的準備使用的圖片集`(一張一張的*.jpg檔案，不是資料夾)`，放到`/content/darknet/custom_yolo/datasets`

In [ ]:
!tar -xf /content/GDrive/MyDrive/Yolo_Data/car_imgs_aug.tar -C /content/darknet/custom_yolo/datasets
!tar -xf /content/GDrive/MyDrive/Yolo_Data/car_annots_aug.tar -C /content/darknet/custom_yolo/datasets

# 2. 準備圖片的標註檔
將圖片做標註，每張圖片產生一個Yolo格式的標註檔案`*.txt`，製作一個`產生含有訓練資料圖片路徑的*.txt檔案`

'
### ★★ 若有需要切割成訓練資料跟測試資料，則變成製作`含有訓練資料圖片路徑的*.txt檔案`及`含有測試資料圖片路徑的*.txt檔案`

```
# 標註txt檔，長得像這樣 <object-class> <x_center> <y_center> <width> <height>

1 0.716797 0.395833 0.216406 0.147222
```

```
#　訓練資料圖片路徑的txt檔案(若有需要測試資料，同樣也需一份txt檔案)， 長成：

<你自己存放訓練圖片資料夾路徑>/img1.jpg
<你自己存放訓練圖片資料夾路徑>/img2.jpg
<你自己存放訓練圖片資料夾路徑>/img3.jpg
```





In [ ]:
"""
當標註格式是Pascal VOC格式的時候，要轉換成Yolo格式
"""

import xml.etree.ElementTree as ET
import os
from glob import glob

DataSets_save_dir_path = "/content/darknet/custom_yolo/datasets"
XML_PATH = DataSets_save_dir_path
convert2txtPATH = DataSets_save_dir_path

CLS = {"car": 0, "license": 1}


#################################################

i = 0
for xml in os.listdir(XML_PATH):
  if "xml" in xml.split("."):
    print(f"第{i}份")
    tree = ET.parse(XML_PATH + "/" + xml)
    root = tree.getroot()
    
    # 找到圖片的高、寬，以利後續計算 BBOX 
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)

    # 對所有的根點做迭代
    with open(convert2txtPATH + "/" + xml.split(".")[0] + ".txt", mode="w", encoding="utf-8") as f:
      for obj in root.iter('object'):
        # 找到原本 xml 標註格式裡面的內容
        CLASS = obj.find("name").text
        CLASS_IDX = CLS[CLASS]
        BBOX = obj.find("bndbox")
        COORDINATE = (int(BBOX.find("xmin").text), int(BBOX.find("ymin").text), int(BBOX.find("xmax").text), int(BBOX.find("ymax").text))

        # 將座標轉換成 Yolo 的 x_center, y_center, w, h
        x_center = round((COORDINATE[2] + COORDINATE[0]) / (2 * width), 6)
        y_center = round((COORDINATE[3] + COORDINATE[1]) / (2 * height), 6)
        w = round((COORDINATE[2] - COORDINATE[0]) / width, 6)
        h = round((COORDINATE[3] - COORDINATE[1]) / height, 6)


        # 寫入最終結果
        f.write(str(CLASS_IDX) + " " + str(x_center) + " " + str(y_center) + " " + str(w) + " " + str(h) + "\n")
      
    i += 1

In [ ]:
"""
圖片確認有上傳後，「需要」切割出測試資料時，請執行此Cell
"""

# 讀取資料夾下面的檔案內容
import os
import random

#　查看存放圖片的資料夾中有什麼內容
DataSets_save_dir_path = "/content/darknet/custom_yolo/datasets"
dir_contents = os.listdir(DataSets_save_dir_path)

# 製作出每張圖片路徑的List
imgs = [content for content in dir_contents if "jpg" in content.split(".")]

# 接著打亂選單，以隨機切割資料 (shuffle功能是return None的功能)
random.shuffle(imgs)

# 看一下有多少資料數
data_counts = len(imgs)


# 製作train.txt (90%的訓練資料)
with open(DataSets_save_dir_path + "/" + "train.txt", mode="w", encoding="utf-8") as f:
  for img in imgs[ : int(data_counts*0.9) ]:
    if img != imgs[ : int(data_counts*0.9) ][-1]:
      f.write(DataSets_save_dir_path + "/" + img + "\n")
    else:
      f.write(DataSets_save_dir_path + "/" + img)

print("train.txt建立完畢")


# 製作test.txt (剩下的)
with open(DataSets_save_dir_path + "/" + "test.txt", mode="w", encoding="utf-8") as f:
  for img in imgs[ int(data_counts*0.9) : ]:
    if img != imgs[ int(data_counts*0.9) : ][-1]:
      f.write(DataSets_save_dir_path + "/" + img + "\n")
    else:
      f.write(DataSets_save_dir_path + "/" + img)

print("test.txt建立完畢")

# 3. 創建自己的`obj.names`檔案，內容為`一行一輸出類別`，最後將檔案放到`/content/darknet/custom_yolo/data`

In [ ]:
# 將你自己物件辨識的標註類別填入到下面 labels 中，然後執行即可
# Classes = [
#           CLASS A,
#           CLASS B,
#           CLASS C,
#           CLASS E,
#           CLASS F
# ]


Classes = [
          "Car",
          "License"
]

ObjName_save_dir_path = "/content/darknet/custom_yolo/data"

with open(ObjName_save_dir_path + "/" + "obj.names", mode="w", encoding="utf-8") as f:
  for Class in Classes:
    if Class != Classes[-1]:
      f.write(Class + "\n")
    else:
      f.write(Class)

print("檔案創建完成")

# 4. 建立自己的`obj.data`檔案，檔案內容如下所示，`其中只要修改classes就好`，最後將此檔案放到`/content/darknet/custom_yolo/data`

**********************************************************************
***** 須注意裡面 train、valid 的路徑要跟`你自己資料集的路徑`一致 *****
**********************************************************************

```
# 檔案內容 (只要修改classes)

classes = 填入你自己的輸出類別數
train  = <存放圖片的資料夾路徑> /train.txt
valid  = <存放圖片的資料夾路徑> /test.txt
names = 填入你自己的obj.names檔案路徑
backup = 填入你模型訓練之後，係數權重檔案weight的存放路徑

```

In [ ]:
classes_num = int(input("你的輸出類別數為？\n"))

DataSets_save_dir_path = "/content/darknet/custom_yolo/datasets"

contents = [
          "classes = {}".format(classes_num),
          "train = {}/train.txt".format(DataSets_save_dir_path),
          "valid = {}/test.txt".format(DataSets_save_dir_path),  
          "names = {}/obj.names".format(ObjName_save_dir_path),
          "backup = /content/GDrive/MyDrive/Yolo_v4_Tiny"
]

ObjData_save_dir_path = ObjName_save_dir_path

with open(ObjData_save_dir_path + "/" + "obj.data", mode="w", encoding="utf-8") as f:
  for content in contents:
    if content != contents[-1]:
      f.write(content + "\n")
    else:
      f.write(content)

print("檔案建立完畢")

# 5. 創建自己的`yolov4-tiny-obj.cfg`檔案
* 可直接複製原本就包含在`/content/darknet/cfg`中的`yolov4-tiny-coutom.cfg`檔案，然後修改裡面的參數內容後，重新改名為`yolov4-tiny-obj.cfg`，將其存放到 `/content/darknet/custom_yolo/cfg`

### 接著修改下載的cfg檔案裡面的參數


  有任何問題詳參https://github.com/AlexeyAB/darknet#how-to-train-tiny-yolo-to-detect-your-custom-objects

### 注意需修改預設 anchors 值 (於219、268行)
  

In [ ]:
# 使用以下指令來獲得你自己的 anchor 值  (注意看原本的 cfg 文件中有幾組 anchor  =>  你的 num_of_clusters 設定就要依據幾組來做設定)
!./darknet detector calc_anchors /content/darknet/custom_yolo/data/obj.data -num_of_clusters 6 -width 416 -height 416

In [ ]:
# 記得 yolov4-tiny-custom.cfg 在設定完後，要改名為 yolov4-tiny-obj.cfg ，然後上傳回要的路徑 /content/darknet_custom_yolo/cfg

# 6. 使用以下指令開始訓練模型
```
./darknet detector train <你的obj.data> <你的yolov4-obj.cfg> <預先訓練好的卷積yolov4.conv.29>
```

In [ ]:
# 開始訓練前請確認你有開啟 GPU、OpenCV 的使用，然後要進行編譯，這些步驟都有做才能訓練模型
# (建議使用) 另外若希望看到模型訓練的Loss圖，請在最後加上 -map ，圖片會存放在 /content/darknet/<chart_設定檔檔名.png>，可以了解你模型訓練的情形為何

!./darknet detector train /content/darknet/custom_yolo/data/obj.data /content/darknet/custom_yolo/cfg/yolov4-tiny-obj.cfg /content/darknet/custom_yolo/yolov4-tiny.conv.29 -dont_show -map


In [ ]:
# 若有中斷，則將權重檔案改成最後一次的，就可以接續上次繼續訓練
!./darknet detector train /content/darknet/build/darknet/x64/data/obj.data /content/darknet/build/darknet/x64/cfg/yolov4-obj.cfg <最後一次權重檔路徑> -dont_show -map

In [ ]:
# 運行以下指令，可以知道你的模型的mAP為何 (預測精準度)， 請注意Overfitting的問題 (必須包含當初你的圖片資料及標註資料)
!./darknet detector map /content/darknet/custom_yolo/data/obj.data /content/darknet/custom_yolo/cfg/yolov4-tiny-obj.cfg /content/yolov4-tiny-obj_best.weights

In [ ]:
# 用你自己訓練好的模型來做預測

!./darknet detector test /content/darknet/custom_yolo/data/obj.data /content/darknet/custom_yolo/cfg/yolov4-tiny-obj.cfg /content/yolov4-tiny-obj_best.weights /content/20210708_6.jpg -dont_show -ext_output -thresh 0.5
imShow("predictions.jpg")